## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, OneHotEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss

## Reading Data

In [2]:
# reading CSVs
vehicle_df = pd.read_csv('../data/localdata/Traffic_Crashes_Vehicles.csv')
people_df = pd.read_csv('../data/localdata/Traffic_Crashes_People.csv')
crash_df = pd.read_csv('../data/localdata/Traffic_Crashes_Crashes.csv')

/Users/michaelromanski/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/michaelromanski/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (20,24,25,26,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Data Cleaning

In [3]:
# Crashes DataFrame

crash_df.drop(columns=['WORK_ZONE_I', 'WORK_ZONE_TYPE', 'DOORING_I', 'WORKERS_PRESENT_I',
                                          'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'RD_NO', 'REPORT_TYPE', 'NOT_RIGHT_OF_WAY_I', 'CRASH_DATE_EST_I'], axis=1, inplace=True)
crash_df.dropna(subset = ['INJURIES_TOTAL', 'LATITUDE', 'MOST_SEVERE_INJURY', 
                          'STREET_DIRECTION', 'BEAT_OF_OCCURRENCE'], axis=0, inplace=True)
crash_df.columns = crash_df.columns.str.title()
crash_df.columns = crash_df.columns.str.replace('_', ' ')

In [4]:
# Vehicles DataFrame

vehicle_df = vehicle_df[['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'CRASH_DATE', 'UNIT_NO', 'UNIT_TYPE',
                       'VEHICLE_YEAR', 'VEHICLE_USE', 'VEHICLE_TYPE', 'MANEUVER', 'OCCUPANT_CNT',
                         'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I', 'AREA_04_I',
                       'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I', 'AREA_09_I', 'AREA_10_I', 'AREA_11_I',
                       'AREA_12_I', 'AREA_99_I', 'FIRST_CONTACT_POINT']]
vehicle_df.dropna(subset = ['VEHICLE_USE', 'FIRST_CONTACT_POINT', 'UNIT_TYPE'], axis=0, inplace=True)
vehicle_df['VEHICLE_YEAR'].fillna(value = 'Unknown', inplace=True)

vehicle_df.columns = vehicle_df.columns.str.title()
vehicle_df.columns = vehicle_df.columns.str.replace('_', ' ')

In [5]:
# People DataFrame

people_df.drop(columns=['RD_NO', 'CELL_PHONE_USE', 'PEDPEDAL_ACTION', 
                        'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'SEAT_NO', 
                        'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'BAC_RESULT', 'BAC_RESULT VALUE', 
                        'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'CITY', 'STATE', 'ZIPCODE']
               , axis=1, inplace=True)

people_df.dropna(subset = ['AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'VEHICLE_ID', 
                           'SAFETY_EQUIPMENT', 'SEX'], axis=0, inplace=True)

people_df.columns = people_df.columns.str.title()
people_df.columns = people_df.columns.str.replace('_', ' ')

## Feature Engineering Crash Score

In [6]:
# Injury Score

crash_df[['Injuries Total', 'Injuries Fatal', 'Injuries Incapacitating', 
            'Injuries Non Incapacitating', 'Injuries Reported Not Evident']][crash_df['Injuries Total']!=0][:50]

crash_df['total injured'] = crash_df['Injuries Fatal'] + crash_df['Injuries Incapacitating'] + crash_df['Injuries Non Incapacitating'] + crash_df['Injuries Reported Not Evident']

# Checking if added up columns in 'total injured' make up 'Injuries Total'
(crash_df['total injured'] == crash_df['Injuries Total']).value_counts()

# Injury Score column is a linear combination of the factors that make up Injuries Total simply multiplied by a constant depending on their severity
crash_df['Injury Score'] = crash_df['Injuries Fatal']*7 + crash_df['Injuries Incapacitating']*3 + crash_df['Injuries Non Incapacitating']*2 + crash_df['Injuries Reported Not Evident']

# Accounting Material Damage of Car and Surrounding Areas )
# Crash Score 

crash_df['Damage'].value_counts()
mapping = {'OVER $1,500': 3, '$501 - $1,500': 2, "$500 OR LESS": 1}
crash_df['Damage_ODE'] = crash_df['Damage'].map(mapping)
crash_df['Injury Score'] = crash_df['Injury Score'] + crash_df['Damage_ODE']*3
crash_df['Crash Score'] = crash_df['Injury Score']

In [7]:
print(crash_df.shape)
print(vehicle_df.shape)

(746352, 43)
(1497524, 25)


## Merging Vehicle and Crash DataFrames

In [8]:
vehicles_crashes_df = vehicle_df.merge(crash_df, on = 'Crash Record Id', how = 'inner')
vehicles_crashes_df.drop_duplicates(subset='Crash Record Id', inplace=True)

In [9]:
categorical = ['Weather Condition', 'Roadway Surface Cond', 'Road Defect', 'Alignment',
                       'Traffic Control Device', 'Device Condition', 'Crash Hour', 'Trafficway Type', 'Maneuver'] #potentiall vehicle defects

ohe = OneHotEncoder(drop='first')
ohe.fit(vehicles_crashes_df[categorical])
categorical_encoded = pd.DataFrame(ohe.transform(vehicles_crashes_df[categorical]).todense(),
                               columns=ohe.get_feature_names())
# for Nick's version of OneHotEncoder, since its newer, the attribute is get_feature_names_out but get_feature_name for bobby and mike's version

In [10]:
area_df = vehicles_crashes_df[['Crash Record Id','Area 00 I', 'Area 01 I', 'Area 02 I', 'Area 03 I', 'Area 04 I',
                       'Area 05 I', 'Area 06 I', 'Area 07 I', 'Area 08 I', 'Area 09 I', 'Area 10 I', 'Area 11 I',
                       'Area 12 I', 'Area 99 I', 'Latitude', 'Longitude']]

In [11]:
vehicles_crashes_df.to_csv('../data/localdata/vehicles_crashes.csv')

In [12]:
area_df.head()

,Crash Record Id,Area 00 I,Area 01 I,Area 02 I,Area 03 I,Area 04 I,Area 05 I,Area 06 I,Area 07 I,Area 08 I,Area 09 I,Area 10 I,Area 11 I,Area 12 I,Area 99 I,Latitude,Longitude
0,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,41.885256,-87.667091
2,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.900043,-87.755577
5,80f5b6eff349a72094c9c7a11745549caa8bd39b57cedd...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,41.999816,-87.663273
7,c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,41.988622,-87.661653
9,522f18b8de9a8a2bb313e7cf54bc60da64bf0085098cd0...,NaN,Y,NaN,NaN,NaN,Y,Y,Y,NaN,NaN,NaN,Y,Y,NaN,41.873096,-87.742362


In [13]:
area_df.shape

(745092, 17)

In [14]:
multi_yes = area_df.columns[area_df.eq('Y').sum() > 1]

In [15]:
mask = area_df[multi_yes].eq('Y').sum(axis=1) <= 1

In [16]:
filtered_area_df = area_df[mask]

In [17]:
filtered_area_df.shape

(483664, 17)

In [18]:
filtered_area_df.head()

,Crash Record Id,Area 00 I,Area 01 I,Area 02 I,Area 03 I,Area 04 I,Area 05 I,Area 06 I,Area 07 I,Area 08 I,Area 09 I,Area 10 I,Area 11 I,Area 12 I,Area 99 I,Latitude,Longitude
2,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.900043,-87.755577
7,c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,41.988622,-87.661653
13,7d0acd72b06cdef52a0784b26b75e8dbff86fa24c63410...,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.830891,-87.627238
15,26aaf4abbd1f8672ceb1510bc56af90e8079b53d52b7ed...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,41.756551,-87.610690
17,169ef212c6902639b42f985403007ef11ed3ed9cbf7177...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,41.899260,-87.694179


In [19]:
for column in filtered_area_df.columns[1:15]:
    mapping = {'Y': 1, 'N': np.nan}
    
    area_column = filtered_area_df[column].map(mapping)
    
    filtered_area_df[column] = area_column
print(filtered_area_df)

<ipython-input-19-beffbba6ad4d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_area_df[column] = area_column


                                           Crash Record Id  Area 00 I  \
2        81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...        NaN   
7        c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...        NaN   
13       7d0acd72b06cdef52a0784b26b75e8dbff86fa24c63410...        NaN   
15       26aaf4abbd1f8672ceb1510bc56af90e8079b53d52b7ed...        NaN   
17       169ef212c6902639b42f985403007ef11ed3ed9cbf7177...        NaN   
...                                                    ...        ...   
1485230  3fd91a7ef43438191bfed3920cdc68868c90283bd6ea7e...        NaN   
1485234  34a300225be49643ff7a6cb00648f55e44101ffdba2799...        NaN   
1485236  bf04349af64331c7cee45a40bf2979ed3d61fdf82456a0...        1.0   
1485241  545044224202f24b78f10eef0aef2935ff9abfc4f10309...        NaN   
1485243  6b53b6a8fb20fc4839856798fb480cd21db71c6a3fa9f9...        NaN   

         Area 01 I  Area 02 I  Area 03 I  Area 04 I  Area 05 I  Area 06 I  \
2              1.0        NaN        NaN      

In [20]:
filtered_area_df.fillna(0, inplace=True)

/Users/michaelromanski/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [21]:
from sklearn.preprocessing import OrdinalEncoder

In [22]:
encoder = OrdinalEncoder()


In [23]:
filtered_area_df.isna().sum()

Crash Record Id    0
Area 00 I          0
Area 01 I          0
Area 02 I          0
Area 03 I          0
Area 04 I          0
Area 05 I          0
Area 06 I          0
Area 07 I          0
Area 08 I          0
Area 09 I          0
Area 10 I          0
Area 11 I          0
Area 12 I          0
Area 99 I          0
Latitude           0
Longitude          0
dtype: int64

In [28]:
def fill_area(row):
    if row["Area 00 I"] == 1:
        return 0
    elif row["Area 01 I"] == 1:
        return 1
    elif row["Area 02 I"] == 1:
        return 2
    elif row["Area 03 I"] == 1:
        return 3
    elif row["Area 04 I"] == 1:
        return 4
    elif row["Area 05 I"] == 1:
        return 5
    elif row["Area 06 I"] == 1:
        return 6
    elif row["Area 07 I"] == 1:
        return 7
    elif row["Area 08 I"] == 1:
        return 8
    elif row["Area 09 I"] == 1:
        return 9
    elif row["Area 10 I"] == 1:
        return 10
    elif row["Area 11 I"] == 1:
        return 11
    elif row["Area 12 I"] == 1:
        return 12
    elif row["Area 99 I"] == 1:
        return 99
    else:
        pass

filtered_area_df['Area Number'] = filtered_area_df.apply(fill_area, axis=1)

filtered_area_df['Area Number']

<ipython-input-28-f52366645d3a>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_area_df['Area Number'] = filtered_area_df.apply(fill_area, axis=1)


2           1.0
7          99.0
13          3.0
15         99.0
17         12.0
           ... 
1485230    12.0
1485234    11.0
1485236     0.0
1485241    99.0
1485243    99.0
Name: Area Number, Length: 483664, dtype: float64

In [30]:
filtered_area_df['Area Number'].value_counts()

99.0    133928
1.0      76671
12.0     50993
2.0      28716
0.0      28128
8.0      22496
11.0     20057
6.0      18053
5.0      17111
3.0      14542
7.0      13473
4.0       9770
9.0       2935
10.0      2335
Name: Area Number, dtype: int64

In [31]:
filtered_area_df.head()

,Crash Record Id,Area 00 I,Area 01 I,Area 02 I,Area 03 I,Area 04 I,Area 05 I,Area 06 I,Area 07 I,Area 08 I,Area 09 I,Area 10 I,Area 11 I,Area 12 I,Area 99 I,Latitude,Longitude,Area Number
2,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.900043,-87.755577,1.0
7,c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,41.988622,-87.661653,99.0
13,7d0acd72b06cdef52a0784b26b75e8dbff86fa24c63410...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.830891,-87.627238,3.0
15,26aaf4abbd1f8672ceb1510bc56af90e8079b53d52b7ed...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,41.756551,-87.610690,99.0
17,169ef212c6902639b42f985403007ef11ed3ed9cbf7177...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,41.899260,-87.694179,12.0


In [32]:
filtered_area_df.to_csv('../data/localdata/filtered_area.csv')